### Import Necessary Packages

In [1]:
import boto3 
import gc
import numpy as np
import sagemaker 
from sagemaker import get_execution_role
import os

### Define Bucket and Prefix for Training and Validation data 

In [2]:
bucket = sagemaker.Session().default_bucket()
prefix = 'demand-prediction'

### Configure Necessary Parameters 

In [3]:
algo_name = 'lightgbm'
ecr_repository_name = algo_name 
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
print(account_id)
print(region)

415955966947
us-east-1


### Build and Push Docker Image to ECR 

In [4]:
%%bash -s "$account_id" "$region"
cd light_gbm_sagemaker/script 
./build_and_push.sh $1 $2 lightgbm


Step 1/7 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
80bce60046fa: Pulling fs layer
55a738a15540: Pulling fs layer
e19cf0706c62: Pulling fs layer
de4cdd6c27d1: Pulling fs layer
de4cdd6c27d1: Waiting
e19cf0706c62: Verifying Checksum
e19cf0706c62: Download complete
55a738a15540: Verifying Checksum
55a738a15540: Download complete
de4cdd6c27d1: Verifying Checksum
de4cdd6c27d1: Download complete
80bce60046fa: Verifying Checksum
80bce60046fa: Download complete
80bce60046fa: Pull complete
55a738a15540: Pull complete
e19cf0706c62: Pull complete
de4cdd6c27d1: Pull complete
Digest: sha256:9775877f420d453ef790e6832d77630a49b32a92b7bedf330adf4d8669f6600e
Status: Downloaded newer image for ubuntu:16.04
 ---> 9ff95a467e45
Step 2/7 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git &&     rm -rf /var/lib/apt/lists/*
 ---> Running in d5f0dce43ed9
Get:1 http://security.ubunt

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Train Model by the Docker Image Just Pushed to ECR

In [5]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)


415955966947.dkr.ecr.us-east-1.amazonaws.com/lightgbm:latest


In [6]:
import sagemaker
import time

job_name = "{}-{}".format(algo_name, str(time.time()))
est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(num_iterations=100)
train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/validation/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'valid': val_config })


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-06-17 13:23:06 Starting - Starting the training job...
2021-06-17 13:23:09 Starting - Launching requested ML instancesProfilerReport-1623936186: InProgress
...
2021-06-17 13:24:05 Starting - Preparing the instances for training.........
2021-06-17 13:25:23 Downloading - Downloading input data...
2021-06-17 13:26:03 Training - Downloading the training image..[LightGBM] [Warning] output_model is set=/opt/ml/model/model.txt, output_model=LightGBM_model.txt will be ignored. Current value: output_model=/opt/ml/model/model.txt
[LightGBM] [Warning] num_iterations is set=100, num_trees=100 will be ignored. Current value: num_iterations=100
[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Construct bin mappers from text data time 0.40 seconds
[LightGBM] [Info] Finished loading data in 6.735252 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.531136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memo

[LightGBM] [Info] Iteration:44, training l2 : 4.52399
[LightGBM] [Info] Iteration:44, valid_1 l2 : 3.84626
[LightGBM] [Info] 16.358537 seconds elapsed, finished iteration 44
[LightGBM] [Info] Iteration:45, training l2 : 4.51561
[LightGBM] [Info] Iteration:45, valid_1 l2 : 3.83609
[LightGBM] [Info] 16.671001 seconds elapsed, finished iteration 45
[LightGBM] [Info] Iteration:46, training l2 : 4.50699
[LightGBM] [Info] Iteration:46, valid_1 l2 : 3.82506
[LightGBM] [Info] 17.117764 seconds elapsed, finished iteration 46
[LightGBM] [Info] Iteration:47, training l2 : 4.4994
[LightGBM] [Info] Iteration:47, valid_1 l2 : 3.81702
[LightGBM] [Info] 17.374242 seconds elapsed, finished iteration 47
[LightGBM] [Info] Iteration:48, training l2 : 4.49193
[LightGBM] [Info] Iteration:48, valid_1 l2 : 3.80879
[LightGBM] [Info] 17.687706 seconds elapsed, finished iteration 48
[LightGBM] [Info] Iteration:49, training l2 : 4.48475
[LightGBM] [Info] Iteration:49, valid_1 l2 : 3.8034
[LightGBM] [Info] 17.9485


2021-06-17 13:27:03 Uploading - Uploading generated training model
2021-06-17 13:27:03 Completed - Training job completed
Training seconds: 101
Billable seconds: 101
